# Deploy an Agent

Once a Genome has been uploaded or created on lab, it is possible to deploy an agent. In the sidebar, select the Genome from the Perception Manifold area. Find the Genome named 'demo-all', next select an agent size, then select 'M' for 'medium'. This will deploy an agent using the Genome topology. On the dashboard page you can see the status of your agent as it is created.

An agent is a computer in the cloud (server, virtual machine) or local machine preconfigured with the GAIuS operating system. On IA's cloud, this is done automatically through the Lab when spawning a new agent.

After a few minutes, a new agent should appear in the 'Deployed Agents' area of the sidebar. Click on your agent. If following along, copy and paste the <b>agent name</b> and the <b>api key</b> into the agent_info below.

# Connect to Your Agent

You have options on how to connect to a running agent. You can use the provided SDK in your preferred language, or roll your own.

### Use the SDK's AgentClient

Install the Python version of the SDK via:

`pip3 install ia-sdk`

or, if using the Jupyter for Intelligent Artifacts container, open a terminal and update it with:

`pip3 install --user -U ia-sdk`

To check the current sdk version:

`pip3 show ia-sdk`

The AgentClient in the SDK requires only the URL and api key.  Once you connect, you can define the ingress and query nodes for the agent.  Each AgentClient can define its own ingress and query nodes.  This allows multiple inputs sources to connect to different ingress nodes of the agent, as well as allowing to query different Cognitive Processors for answers.

For example, to connect to a cloud agent:

~~~

# update your agent information, and then connect to it
api_key = '<secret-api-key-here>'
name    = '<bottle-name-here>'
domain  = 'intelligent-artifacts.com'
secure  = True

from ia.gaius.agent_client import AgentClient

agent_info = {'api_key': api_key,
               'name':  name,
               'domain': domain,
               'secure': secure}
agent = AgentClient(agent_info)
agent.connect()
agent

~~~

To connect to a local agent:

~~~

from ia.gaius.agent_client import AgentClient

agent_info = {'api_key': 'ABCD-1234',
               'name':  '',
               'domain': '<agent>:<port>',
               'secure': False}
agent = AgentClient(agent_info)
agent.connect()
agent

~~~
For local agents it may be necessary to connect to the docker network, if connecting via another docker container

### Or, roll your own connections
To roll your own, you simply need the agent's host and domain names, the secret API key, and the IDs of each primitive you'd like to connect.  You can use a library like Python's `requests` to POST data and query the primitives.

For example:

Let's connect to either a local or a cloud agent

~~~
import requests

# Local Agent Info
name    = 'agent'
api_key = 'ABCD-1234'

# connect to local agent with http 
requests.get(f"http://{name}/connect", 
             headers={'X-API-KEY': api_key}).json()
             
# Cloud Agent Info
name    = '<agent-name-here>'
domain  = 'intelligent-artifacts.com'
api_key = '<secret-api-key-here>'

# connect to a cloud agent with https and domain
requests.get(f"https://{name}.{domain}/connect", 
             headers={'X-API-KEY': api_key}).json()
~~~

To observe data:

In order to observe data you need to have the primitive ids of the Cognitive Processors. If you don't know the ids, you can retrieve them by connecting to the agent with request first.

The following examples are for cloud bottles.

~~~
# data as a GDF you want to observe
data = {"strings": ["hello","world"], "vectors": [], "emotives": {}}

agent_connection_info = requests.get(f"https://{name}.{domain}/connect", 
             headers={'X-API-KEY': api_key}).json()

# navigate into the bottle_connection_info dictionary to get primitive ids from genome data
primitive_id = agent_connection_info['genome']['elements']['nodes'][0]['data']['id']

# observe data
requests.post(f"https://{name}.{domain}/{primitive_id}/observe", 
              headers={'X-API-KEY': api_key}, 
              json={'data': data}).json()
~~~

To learn data:

After you've observed a few events

~~~

requests.post(f"https://{name}.{domain}/{primitive_id}/learn", 
              headers={'X-API-KEY': api_key}).json()

~~~

To query for answers:

After you've observed additional data, one can try to get predictions

~~~

# data as a GDF you want to observe in order to get predictions
data = {"strings": ["hello"], "vectors": [], "emotives": {}}

# observe data
requests.post(f"https://{name}.{domain}/{primitive_id}/observe", 
              headers={'X-API-KEY': api_key}, 
              json={'data': data}).json()

requests.post(f"https://{name}.{domain}/{primitive_id}/predictions", 
              headers={'X-API-KEY': api_key}).json()
~~~

To get a fresh start, one can

clear all memory of the Cognitive Processor:



~~~
              
requests.post(f"https://{name}.{domain}/{primitive_id}/clear-all-memory", 
              headers={'X-API-KEY': api_key}).json()              
~~~

In [1]:
# The sdk version used in this tutorial
!pip3 show ia-sdk

Name: ia-sdk
Version: 0.3.2
Summary: SDK for Intelligent Artifact's GAIuS agents.
Home-page: https://intelligent-artifacts.com
Author: Intelligent Artifacts
Author-email: support@intelligent-artifacts.com
License: None
Location: /home/jupyter/.local/lib/python3.8/site-packages
Requires: plotly, requests, pymongo
Required-by: 


In [2]:
# update your agent information, and then connect to it
from ia.gaius.agent_client import AgentClient

api_key = '<YOUR-API-KEY-HERE>'
name    = '<YOUR-AGENT-NAME-HERE>'
domain  = 'intelligent-artifacts.com'
secure  = True

agent_info = {'api_key': api_key,
               'name':  name,
               'domain': domain,
               'secure': secure}

In [3]:
agent_info = {'api_key': 'ABCD-1234',
              'name': '',
              'domain': 'gaius-api:80',
              'secure': False}

In [4]:
agent = AgentClient(agent_info)
agent.connect()
agent

<.gaius-api:80| secure: False, connected: True, gaius_agent: simple,                   ingress_nodes: 0, query_nodes: 0>

In [5]:
# Let's use only one node
ingress_nodes = ['P1']
query_nodes   = ['P1']

agent.set_ingress_nodes(ingress_nodes)
agent.set_query_nodes(query_nodes)

[{'id': 'p46b6b076c', 'name': 'P1'}]

You can visualize the Genome for the connected agent by calling `agent.genome.display()`

This functionality helps visualize larger topologies

In [6]:
agent.genome.display()

Cytoscape(background='white', data={'agent': 'simple', 'boxSelectionEnabled': True, 'description': '', 'elemen…